In [35]:
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
sys.path.append('../code/')
%matplotlib inline
import itertools
from pathlib import Path
from torch.autograd import Variable


In [44]:
test_error = {}
dataset =3
gid = 0
for fold_num in range(5):
    test_error[fold_num] = {}
    for appliance in ['hvac', 'fridge', 'dr', 'dw', 'mw']:
        test_error[fold_num][appliance] = {}
        for lr in [0.001, 0.01, 0.1]:
            test_error[fold_num][appliance][lr] = {}
            for iters in [200000]:
            
                directory = "../code/baseline/cnn-tree/{}/{}/{}/{}/0.0/".format(dataset, fold_num, lr, iters)
                filename = "test-error-[\'{}\'].npy".format(appliance)
                
                full_path = directory + filename
                my_file = Path(full_path)
                if not my_file.exists():
                    print("CUDA_VISIBLE_DEVICES={} python cnn-tree.py {} {} {} 0 {} {} &".format(gid, dataset, lr, iters, fold_num, appliance))
                    gid += 1
                    if gid == 4:
                        gid = 0
                else:
                    for it in range(1000, 200000+1, 1000):
                        test_error[fold_num][appliance][lr][it] = np.load(full_path).item()[it][appliance]

In [47]:
best_param = {}
for appliance in ['hvac', 'fridge', 'dr', 'dw', 'mw']:
    best_param[appliance] = {}
    for fold_num in range(5):
        best_param[appliance][fold_num] = {}
        
for appliance in ['hvac', 'fridge', 'dr', 'dw', 'mw']:
    for fold_num in range(5):
        error = np.inf
        for lr in [0.001, 0.01, 0.1]:
            for iters in range(1000, 200000+1, 1000):
                if error > test_error[fold_num][appliance][lr][iters]:
                    error = test_error[fold_num][appliance][lr][iters]
                    best_param[appliance][fold_num]['lr'] = lr
                    best_param[appliance][fold_num]['iters'] = iters

In [48]:
np.save("../code/baseline/result/cnn-individual-param-{}.npy".format(dataset), best_param)

In [49]:
best_error = {}
for appliance in ['hvac', 'fridge', 'dr', 'dw', 'mw']:
    best_error[appliance] = {}
    for fold_num in range(5):
        lr = best_param[appliance][fold_num]['lr']
        iters = best_param[appliance][fold_num]['iters']
        best_error[appliance][fold_num] = test_error[fold_num][appliance][lr][iters]

In [50]:
best_error

{'dr': {0: 51.275147358484482,
  1: 63.117866459622157,
  2: 71.019345774617292,
  3: 82.201023565871481,
  4: 64.850601715510891},
 'dw': {0: 11.024307778145825,
  1: 16.477256485535928,
  2: 19.537914530989642,
  3: 20.592082563342078,
  4: 9.510532094869184},
 'fridge': {0: 28.691444943871449,
  1: 34.489788259617846,
  2: 27.597580598198437,
  3: 32.146306601088391,
  4: 30.844945718913817},
 'hvac': {0: 285.10919817676302,
  1: 264.34939483723429,
  2: 297.90492642833084,
  3: 340.17984514227049,
  4: 302.8673260587405},
 'mw': {0: 7.2305527064629729,
  1: 10.303584358028292,
  2: 6.3609311636474954,
  3: 5.5356631035441675,
  4: 12.339492953294695}}

In [5]:
for appliance in ['hvac', 'fridge', 'dr', 'dw', 'mw']:
    print(appliance, pd.DataFrame(test_error[0][appliance]).min().min())

hvac 331.135803223
fridge 29.0272903442
dr 51.2722740173
dw 11.0249710083
mw 7.21105957031


In [21]:
error = {}
for fold_num in range(5):
    error[fold_num] = {}
    
for appliance in ['hvac', 'fridge', 'dr', 'dw', 'mw']:
    
    

51.272625